In [ ]:
import os
import psycopg2
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Load environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Ensure the API key is correctly loaded
if not openai_api_key:
    raise ValueError("API key for OpenAI not found. Please set it in the .env file.")

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Initialise LangChain with OpenAI
llm = OpenAI(api_key=openai_api_key)

# Define a simple prompt template for converting text to SQL
prompt_template = """
Convert the following text to SQL query for PostgreSQL:
{text}

SQL Query:
"""

# Create a PromptTemplate object
prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

# Create a chain
chain = (
    {"text": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

# Define a function to convert text to SQL and execute the query
def text_to_sql(text):
    # Use LangChain to generate SQL from text
    sql_query = chain.invoke(text)
    
    print(f"Generated SQL query: {sql_query}")  # Print the generated query for debugging
    
    try:
        # Execute the SQL query
        cursor.execute(sql_query)
        # Fetch and return the results
        results = cursor.fetchall()
        return results
    except Exception as e:
        return f"Error executing query: {str(e)}"

# Example prompts to test
prompts = [
    "Show all unique user_ids in the activity table",
]

# Convert the prompts to SQL and execute the queries
for prompt_text in prompts:
    print(f"\nExecuting prompt: {prompt_text}")
    results = text_to_sql(prompt_text)
    print(f"Results: {results}")

# Close the database connection
cursor.close()
conn.close()